In [1]:
# If you run this notebook on Google Colaboratory, uncomment the below to install automl_alex.
#!pip install -q -U automl_alex

In [1]:
import sklearn
import pandas as pd
import time
import automl_alex
from automl_alex import DataPrepare
from automl_alex import Stacking, StackingClassifier, StackingRegressor
print('AutoML-Alex version:', automl_alex.__version__)

AutoML-Alex version: 1.3.8


In [2]:
RANDOM_SEED = 42

# Classifier

## Data

In [3]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
# https://www.openml.org/d/179
dataset = fetch_openml(data_id=179, as_frame=True)
dataset.target = dataset.target.astype('category').cat.codes
dataset.data.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native-country
0,2,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,1,0,2,United-States
1,3,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,0,United-States
2,2,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,2,United-States
3,3,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,2,United-States
4,1,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,2,Cuba


In [4]:
X_train, X_test, y_train, y_test = train_test_split(dataset.data, 
                                                    dataset.target,
                                                    test_size=0.2, 
                                                    random_state=RANDOM_SEED,)
X_train.shape, X_test.shape

((39073, 14), (9769, 14))

## Stacking

In [5]:
model = StackingClassifier(
    clean_and_encod_data=True,
    opt_data_prepare=True,
    models_names = ['LightGBM', 'ExtraTrees', 'LinearModel', 'RandomForest',],
    cat_encoder_names= [
            "HelmertEncoder",
            "OneHotEncoder",
            "CountEncoder",
            "HashingEncoder",
            "BackwardDifferenceEncoder",
        ],
    target_encoders_names= [
            "TargetEncoder",
            "JamesSteinEncoder",
            "CatBoostEncoder",
        ],
    clean_outliers = [True, False],
    num_generator_select_operations = True,
    num_generator_operations = ["/", "*", "-"],
    auto_parameters=True,
    feature_selection=False,
    random_state=RANDOM_SEED,)

18:58:06 | metric is None! Default metric will be used. classifier: AUC, regression: MSE


In [6]:
history = model.opt(X_train, y_train, timeout=600, verbose=3)

18:58:06 | ##################################################
18:58:07 | > Step 1: calc parameters and pruned score: get test 10 trials
ALL MODELS MEAN Score: 0.9091, std: 0.00145
TOP10 MODELS MEAN Score: 0.9091, std: 0.00145
##################################################
ALL MODELS MEAN Score: 0.9098, std: 0.00095
TOP10 MODELS MEAN Score: 0.9098, std: 0.00095
##################################################
ALL MODELS MEAN Score: 0.9103, std: 0.0007
TOP10 MODELS MEAN Score: 0.9103, std: 0.0007
##################################################
ALL MODELS MEAN Score: 0.91, std: 0.00055
TOP10 MODELS MEAN Score: 0.91, std: 0.00055
##################################################
ALL MODELS MEAN Score: 0.9114, std: 0.0013
TOP10 MODELS MEAN Score: 0.9114, std: 0.0013
##################################################
ALL MODELS MEAN Score: 0.9114, std: 0.00105
TOP10 MODELS MEAN Score: 0.9114, std: 0.00105
##################################################
ALL MODELS MEAN Score: 0.9

AttributeError: 'tuple' object has no attribute 'estimator'

In [7]:
model.history_trials_dataframe

,trial_number,model_score,score_std,model_name,model_param,columns,folds,score_folds,y_pred_fold_0,y_pred_fold_1,y_pred_fold_2
29,0,0.9147,0.001327,LightGBM,"{'random_seed': 42, 'num_iterations': 300, 've...",None,7,3,"[0.2947912706787872, 0.6408696078733724, 0.348...","[0.8636942399973465, 0.9944626629172326, 0.403...","[0.27375135912269233, 0.92143763061316, 0.0533..."
26,28,0.9147,0.001327,LightGBM,"{'random_seed': 42, 'num_iterations': 300, 've...",None,7,3,"[0.2947912706787872, 0.6408696078733724, 0.348...","[0.8636942399973465, 0.9944626629172326, 0.403...","[0.27375135912269233, 0.92143763061316, 0.0533..."
28,30,0.9146,0.001605,LightGBM,"{'random_seed': 42, 'num_iterations': 300, 've...",None,7,3,"[0.29490974351468585, 0.648356590416068, 0.362...","[0.876331220849079, 0.9951188203506157, 0.4000...","[0.27296606877655977, 0.9091873873127332, 0.05..."
27,29,0.9146,0.001302,LightGBM,"{'random_seed': 42, 'num_iterations': 300, 've...",None,7,3,"[0.28820996266175397, 0.6429942373345546, 0.35...","[0.8736047449598587, 0.9942296708715281, 0.395...","[0.2653680315318857, 0.9084161001031708, 0.047..."
23,25,0.9145,0.001389,LightGBM,"{'random_seed': 42, 'num_iterations': 300, 've...",None,7,3,"[0.28720455266892964, 0.6398578779102114, 0.36...","[0.8762826225432682, 0.99425581010214, 0.40655...","[0.2656387934682116, 0.9118932823868392, 0.057..."
16,18,0.9145,0.001389,LightGBM,"{'random_seed': 42, 'num_iterations': 300, 've...",None,7,3,"[0.29212539354342254, 0.6552168001215664, 0.33...","[0.8528324618740729, 0.9946954024474262, 0.414...","[0.2776528474545949, 0.9040526507422194, 0.055..."
25,27,0.9144,0.001639,LightGBM,"{'random_seed': 42, 'num_iterations': 300, 've...",None,7,3,"[0.2861682683411354, 0.6194171940467799, 0.373...","[0.7382277485500333, 0.9958456230286437, 0.373...","[0.255733966698499, 0.927889734940262, 0.06549..."
24,26,0.9144,0.001406,LightGBM,"{'random_seed': 42, 'num_iterations': 300, 've...",None,7,3,"[0.2848615228430659, 0.6421341468086584, 0.350...","[0.8825215972594904, 0.9943551509840027, 0.408...","[0.2692785658012025, 0.912814533535276, 0.0547..."
11,12,0.9142,0.001173,LightGBM,"{'random_seed': 42, 'num_iterations': 300, 've...",None,7,3,"[0.30290325738092483, 0.6592076773438541, 0.35...","[0.5996874528110255, 0.9956006996054263, 0.408...","[0.262166713892124, 0.8860454860792766, 0.0420..."
13,15,0.9142,0.002210,LightGBM,"{'random_seed': 42, 'num_iterations': 300, 've...",None,7,3,"[0.2933998299259204, 0.6001209372331096, 0.361...","[0.7606375808282824, 0.9964298175199703, 0.423...","[0.2676067437994037, 0.9192836052458169, 0.034..."


In [ ]:
pd.DataFrame(self.history_trials).sort_values('model_score', ascending=True)

In [7]:
predicts = model.predict(X_test)

In [8]:
print('Test AUC: ', round(sklearn.metrics.roc_auc_score(y_test, predicts),4))

Test AUC:  0.9122


In [9]:
model.best_model_name

'LightGBM'

In [10]:
model.best_model_param

{'random_seed': 42,
 'num_iterations': 300,
 'verbose': -1,
 'device_type': 'cpu',
 'objective': 'binary',
 'num_leaves': 4,
 'learning_rate': 0.2104512193446279}

In [ ]:
# select Features 
# if feature_selection=True
#model.select_columns[:10]

In [22]:
model.study.best_params

{'*': True,
 '-': True,
 '/': True,
 'BackwardDifferenceEncoder': False,
 'CatBoostEncoder': True,
 'CountEncoder': True,
 'HashingEncoder': False,
 'HelmertEncoder': True,
 'JamesSteinEncoder': True,
 'OneHotEncoder': False,
 'TargetEncoder': False,
 'de_clean_outliers': True,
 'lgbm_learning_rate': 0.2104512193446279,
 'lgbm_num_leaves': 4,
 'model_name': 'LightGBM'}

In [35]:
history.sort_values('value', ascending=False).head(5)

,number,value,datetime_start,datetime_complete,duration,params_*,params_-,params_/,params_BackwardDifferenceEncoder,params_CatBoostEncoder,...,params_TargetEncoder,params_de_clean_outliers,params_ext_max_depth,params_ext_min_samples_split,params_lgbm_learning_rate,params_lgbm_num_leaves,params_model_name,params_rf_max_depth,params_rf_min_samples_split,state
12,12,0.9131,2021-03-06 17:51:04.898592,2021-03-06 17:51:18.432278,0 days 00:00:13.533686,True,True,True,False,True,...,False,True,NaN,NaN,0.210451,4.0,LightGBM,NaN,NaN,COMPLETE
8,8,0.9116,2021-03-06 17:50:14.236088,2021-03-06 17:50:42.930326,0 days 00:00:28.694238,True,False,True,False,True,...,False,True,NaN,NaN,0.012886,44.0,LightGBM,NaN,NaN,COMPLETE
0,0,0.9078,2021-03-06 17:48:07.752162,2021-03-06 17:48:26.015151,0 days 00:00:18.262989,False,False,True,True,True,...,True,False,NaN,NaN,NaN,NaN,RandomForest,80.0,53.0,COMPLETE
5,5,0.9066,2021-03-06 17:49:15.035773,2021-03-06 17:49:17.897428,0 days 00:00:02.861655,False,True,False,False,False,...,False,False,NaN,NaN,NaN,NaN,RandomForest,10.0,36.0,COMPLETE
9,9,0.9061,2021-03-06 17:50:42.997106,2021-03-06 17:50:51.803024,0 days 00:00:08.805918,False,False,False,False,True,...,True,True,NaN,NaN,NaN,NaN,RandomForest,10.0,36.0,COMPLETE


In [29]:
model.study.trials[8].params

{'model_name': 'LightGBM',
 'lgbm_num_leaves': 44,
 'lgbm_learning_rate': 0.012886065671894011,
 'HelmertEncoder': True,
 'OneHotEncoder': True,
 'CountEncoder': False,
 'HashingEncoder': True,
 'BackwardDifferenceEncoder': False,
 'TargetEncoder': False,
 'JamesSteinEncoder': False,
 'CatBoostEncoder': True,
 'de_clean_outliers': True,
 '/': True,
 '*': True,
 '-': False}

In [24]:
model = model.get_model_from_iter(X_train, y_train, model.study.trials[8].params)

Finished loading model, total used 300 iterations
Finished loading model, total used 300 iterations
Finished loading model, total used 300 iterations
Finished loading model, total used 300 iterations
Finished loading model, total used 300 iterations
Finished loading model, total used 300 iterations
Finished loading model, total used 300 iterations


In [25]:
predicts = model.predict(X_test)
print('Test AUC: ', round(sklearn.metrics.roc_auc_score(y_test, predicts),4))

Test AUC:  0.9097


## Save & Load

In [26]:
model.save('model_1')

Save model


In [27]:
model_new = BestSingleModelClassifier(random_state=RANDOM_SEED,)
model_new = model_new.load('model_1')

18:03:27 | Load DataPrepare
18:03:28 | Load Model
Finished loading model, total used 300 iterations
18:03:28 | Load Model
Finished loading model, total used 300 iterations
18:03:28 | Load Model
Finished loading model, total used 300 iterations
18:03:28 | Load Model
Finished loading model, total used 300 iterations
18:03:28 | Load Model
Finished loading model, total used 300 iterations
18:03:28 | Load Model
Finished loading model, total used 300 iterations
18:03:28 | Load Model
Finished loading model, total used 300 iterations
Load CrossValidation
Load model


In [28]:
predicts = model_new.predict(X_test)
print('Test AUC: ', round(sklearn.metrics.roc_auc_score(y_test, predicts),4))

Test AUC:  0.9097
